In [ ]:

# Run this cell to import pyspark and define start_spark(), stop_spark(), and supporting helpers

import findspark
findspark.init()

import getpass
import pandas
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Constants used to interact with Azure Blob Storage
global username
username = re.sub('@.*', '', getpass.getuser())

# Utility functions
def dict_to_html(d):
    html = [f'<table width="100%" style="width:100%; font-family: monospace;">']
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')
    return ''.join(html)

def show_as_html(df, n=20):
    display(df.limit(n).toPandas())

def display_spark():
    if 'spark' in globals() and 'sc' in globals():
        name = sc.getConf().get("spark.app.name")
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>. Look for <code>{name}</code> in the Spark UI.</p>',
            f'<ul><li><a href="http://localhost:{sc.uiWebUrl.split(":")[-1]}" target="_blank">Spark UI</a></li></ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul><li>The spark session <code>spark</code> and spark context <code>sc</code> are defined by <code>start_spark()</code>.</li>',
            f'<li>Run <code>stop_spark()</code> before closing the notebook.</li></ul>'
        ]
        display(HTML(''.join(html)))
    else:
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>. Check the completed applications in Spark UI.</p>',
            f'<ul><li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li></ul>'
        ]
        display(HTML(''.join(html)))

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    global spark
    global sc
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)
    spark = (
        SparkSession.builder
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{username}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.driver.memory", f'{master_memory}g')
        .config("spark.executor.memory", f'{worker_memory}g')
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.kubernetes.container.image", "madsregistry001.azurecr.io/hadoop-spark:v3.3.5-openjdk-8")
        .config("spark.kubernetes.container.image.pullPolicy", "IfNotPresent")
        .config("spark.kubernetes.memoryOverheadFactor", "0.3")
        .config("spark.memory.fraction", "0.1")
        .config("spark.app.name", f"{username} (notebook)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    display_spark()

def stop_spark():
    global spark
    global sc
    if 'spark' in globals() and 'sc' in globals():
        spark.stop()
        del spark
        del sc
    display_spark()

# Improve readability of Spark output
display(HTML(r"""<style>
pre { white-space: pre !important; }
table.dataframe td { white-space: nowrap !important; }
table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }
</style>"""))


In [ ]:
# Create, distribute, and wrap data by hand

schema = StructType([
    StructField("Name"       ,  StringType() , True),
    StructField("Department" ,  StringType() , True),
    StructField("Age"        , IntegerType() , True),
    StructField("Gender"     ,  StringType() , True),
    StructField("Salary"     ,  DoubleType() , True)
])
data = spark.createDataFrame(  # Finally, wrap the RDD with metadata by creating a DataFrame = Dataset[Row]
    sc.parallelize(  # Second, take that list of pyspark row objects, distribute them as Spark rows in an RDD[Row]
        [  # First, define a list of pyspark row objects (this is just a Python list in memory on the master node)
            Row("Alpha One"   , "X" , 28 , "M"  ,  80000.0),
            Row("Bravo Two"   , "X" , 25 , "M"  ,  70000.0),
            Row("Charlie"     , "X" , 23 , "M"  ,  80000.0),  # Charlie has no last name, duplicate salary in department X
            Row("Delta Four"  , "Y" , 30 , None , 100000.0),  # Gender is none
            Row("Echo Five"   , "Y" , 27 , "F"  , 120000.0),
            Row("Foxtrot Six" , "Z" , 20 , "F"  ,  90000.0),
            Row("Golf Seven"  , "Z" , 20 , "F"  ,  50000.0),  # Duplicate age in department Z
            Row("Hotel Eight" , "Z" , 38 , "F"  , 100000.0),
            Row("Indigo Nine" , "Z" , 50 , "M"  ,  70000.0),
            Row("Juliet Ten"  , "Z" , 18 , "F"  ,     None),  # Salary is none
        ]
    ), schema=schema)

print(type(data))
data.printSchema()
print(data)
show_as_html(data)

In [ ]:
# Create, distribute, and wrap additional department data by hand

department_schema = StructType([
    StructField("Department", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Campus", StringType(), True)
])
department_data = spark.createDataFrame(
    sc.parallelize(
        [
            Row("X", "Xray",   "U"),
            Row("Y", "Yankee", "V"),
            Row("Z", "Zulu",   "W"),
        ]
    ), schema=department_schema)

print(type(department_data))
department_data.printSchema()
print(department_data)
show_as_html(department_data)

In [ ]:
# Run this cell to start a spark session in this notebook

start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1)